In [47]:
from XMark import XMark_benchmark
import networkx as nx
import numpy as np
import random

In [52]:
def homophily (graph, label):
    similar = [(u, v) for u, v in graph.edges() if graph.nodes[u][label] == graph.nodes[v][label]]
    obs = len(similar) / len(graph.edges())
    
    similar_random = 0
    for u in graph.nodes():
        v = random.choice(list(graph.nodes()))
        if graph.nodes[u][label] == graph.nodes[v][label]:
            similar_random += 1
            
    exp = similar_random / len(graph.nodes())
    
    return obs, exp

def sse(coms, graph, label):
    lab_nod_coms = []
    for c in coms:
        part = []
        for el in c:
            part.append(graph.nodes[el][label])
        lab_nod_coms.append(part)
        
    sse = []
    tot_centr = []
    for c in lab_nod_coms:
        centroid = np.mean(c)
        tot_centr.append(centroid)
        sse_c = 0
        for n in c:
            sse_c += abs( (n - centroid)**2 )
        sse.append(sse_c) 
        
    return sum(sse)

### Categorical attributes

In [53]:
N = 2000
gamma = 3
beta = 2
#studied in the paper, number of labels equal to number of communities
#m_cat = ["auto", "auto"]
m_cat = [2, 5]
theta = 0.3
mu = 0.3
avg_k = 10
min_com = 20

g = XMark_benchmark(N, gamma, beta, mu,
          labels=m_cat,
          noise=theta,
          average_degree=avg_k, min_community=min_com,
          type_attr="categorical")

obs, exp = homophily(g, 'label_0')
print("observed homophily: ", obs)
print("expected homophily: ", exp)
r = nx.attribute_assortativity_coefficient(g, 'label_0')
print("")
print("categorical newman's coefficient: ", r)

observed homophily:  0.6691703358370025
expected homophily:  0.495

categorical newman's coefficient:  0.3294333884349014


### Continuous attributes

In [63]:
N = 2000
gamma = 3
beta = 2
#studied in the paper, number of labels equal to number of communities
#m_cont = ["auto", "auto"]
m_cont = [2, 5]
sigma = 0.5
mu = 0.3
avg_k = 10
min_com = 20

g = XMark_benchmark(N, gamma, beta, mu,
          labels=m_cont,
          std=sigma,
          average_degree=avg_k, min_community=min_com,
          type_attr="continuous")

coms = {frozenset(g.nodes[v]['community']) for v in g}
coms = [list(c) for c in coms]

val_sse1 = sse(coms, g, 'label_0')
print("SSE 1: ", val_sse1)
val_sse2 = sse(coms, g, 'label_1')
print("SSE 2: ", val_sse2)

SSE 1:  498.72875975256676
SSE 2:  518.6653629993475
